In [10]:
import os
import glob
import numpy as np
import networkx as nx
import pandas as pd

###############################################################################
# 1) COMPUTE WHOLE-GRAPH COEVOLUTIONARY ENERGY
###############################################################################
def compute_coevolutionary_energy(G):
    """
    Compute E = - sum_{(u,v)} [fALFF[u]*weight[u,v]*fALFF[v]] 
    skipping edges if any relevant value is None or NaN.
    """
    energy_sum = 0.0
    for u, v, data in G.edges(data=True):
        falff_u = G.nodes[u].get('fALFF', None)
        falff_v = G.nodes[v].get('fALFF', None)
        w = data.get('weight', None)

        # Skip if missing or NaN
        if falff_u is None or falff_v is None or w is None:
            continue
        if any(np.isnan([falff_u, falff_v, w])):
            continue

        energy_sum += falff_u * w * falff_v

    return -energy_sum

###############################################################################
# 2) PARSE FILENAME: E.G. "Pitt_0050007_network.graphml" => (center="Pitt", subject="0050007")
###############################################################################
def parse_filename(filename):
    """
    Given a filename like 'Pitt_0050007_network.graphml',
    extract center='Pitt' and subject='0050007'.
    Adjust if your naming pattern is different.
    """
    base = os.path.basename(filename)
    # e.g. "Pitt_0050007_network.graphml"
    # remove extension
    no_ext = os.path.splitext(base)[0]
    # split on underscore
    parts = no_ext.split('_')  # ["Pitt", "0050007", "network"]
    if len(parts) < 2:
        # fallback
        center = "Unknown"
        subject_id = no_ext
    else:
        center = parts[0]
        subject_id = parts[1]
    return center, subject_id

###############################################################################
# 3) IDENTIFY OUTLIERS USING IQR
###############################################################################
def identify_outliers_iqr(values, iqr_factor=1.5):
    """
    Returns a boolean mask of outliers, 
    where outlier is below Q1 - iqr_factor*IQR or above Q3 + iqr_factor*IQR.
    """
    if len(values) < 2:
        return np.zeros(len(values), dtype=bool)

    q1 = np.percentile(values, 25)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1
    lower_bound = q1 - iqr_factor * iqr
    upper_bound = q3 + iqr_factor * iqr

    return (values < lower_bound) | (values > upper_bound)

###############################################################################
# 4) MAIN
###############################################################################
def main():
    # Directories containing GraphML files for both ASD and Control
    # If you keep them in one directory, you can just point to that directory
    network_dir = r"D:\Project_001\NETWORK_FINAL\ASDNetworks\Partially_Full"
    # Or you can define separate dirs if you prefer:
    # asd_dir = r"D:\Project_001\ASDNetworks"
    # ctrl_dir = r"D:\Project_001\ControlNetworks"

    # Gather all .graphml
    graph_files = glob.glob(os.path.join(network_dir, "*.graphml"))
    if not graph_files:
        print("No graphml files found in:", network_dir)
        return

    # We'll store results in a list of dict: {center, subject, energy}
    results = []

    for gf in graph_files:
        center, subject_id = parse_filename(gf)
        G = nx.read_graphml(gf)
        E = compute_coevolutionary_energy(G)
        if not np.isnan(E):
            results.append({
                "center": center,
                "subject": subject_id,
                "energy": E
            })
        else:
            # If it's NaN, skip or store anyway
            print(f"Energy is NaN for {gf}, skipping.")
            # You could store a row with E=NaN if you want
            # results.append({"center": center, "subject": subject_id, "energy": np.nan})

    if not results:
        print("No valid energies computed. Exiting.")
        return

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["center","subject","energy"])
    print(f"Total subjects with valid energy: {len(df)}")

    # Save all energies
    df.to_csv("all_energies.csv", index=False)

    # Outlier detection (IQR-based) across entire 'energy' distribution
    energies = df["energy"].values
    outlier_mask = identify_outliers_iqr(energies, iqr_factor=1.2)

    # Mark outliers
    df["is_outlier"] = outlier_mask
    outliers_df = df[df["is_outlier"]==True]

    print(f"Outliers found: {len(outliers_df)}")
    if not outliers_df.empty:
        outliers_df.to_csv("energy_outliers.csv", index=False)
        print("Outlier rows saved to 'energy_outliers.csv'")
    else:
        print("No outliers found by this criterion.")

    # Show summary
    print("\n=== Summary of energies ===")
    print(df.describe())

    # Optionally print or inspect outliers
    if not outliers_df.empty:
        print("\n=== Outlier Examples ===")
        print(outliers_df)
 
if __name__ == "__main__":
    main()


Total subjects with valid energy: 93
Outliers found: 4
Outlier rows saved to 'energy_outliers.csv'

=== Summary of energies ===
           energy
count   93.000000
mean   -25.599241
std     29.987012
min   -103.601264
25%    -42.150685
50%    -15.877731
75%     -2.238220
max     18.207431

=== Outlier Examples ===
     center  subject      energy  is_outlier
15   Leuven  0050693 -100.732410        True
19   Leuven  0050697  -99.787804        True
58  Trinity  0050232 -103.601264        True
62  Trinity  0050241  -98.289969        True
